## Lotka-Volterra 1 predator 1 prey test

### Roe Deer and Wolf

In [160]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import *
from random import randint

# Roe deer and wolf
# alpha1 = 9/11 # Birth rate roe deer
# beta1 = 48.6/365.25 # Death rate roe deer
# gamma1 =  25/26 # Death rate wolf
# delta1 = 0.0356*0.7 # Grow rate wolf based on roe deer
# epsilon1 = 0.0356*0.3 # Grow rate predator based on wild board
# eta1 = 6.22 # Brith rate wild boar
# zeta1 = 12.4/182.75 # Death rate wild boar

alpha1 = 0.6 # Birth rate roe deer
beta1 = 0.4 # Death rate roe deer
gamma1 = 0.8 # Death rate wolf
delta1 = 0.4 # Grow rate wolf based on roe deer
epsilon1 = 0.7 # Grow rate predator based on wild board
eta1 = 0.3 # Brith rate wild boar
zeta1 = 0.3 # Death rate wild boar

# Time * step = amount of years
time = 36530 # Approx. amount of days in 100 years
step1 = 1/365.25 # 1 day

pred_name = 'wolf'
prey1_name = 'roe deer'

# Initial populations
init_pred = 10
init_prey1 = 110
init_prey12 = 20

# Differential equation predator population
def pred_eq(gamma, delta, epsilon, w, r, z, step):
    return step * w * (-gamma + delta*r + epsilon*z) 

# Differential equation prey1 population
def prey1_eq(alpha, beta, w, r, step):
    return step * r * (alpha - beta*w)

# Differential equation prey2 population
def prey2_eq(eta, zeta, w, z, step):
    return step * z * (eta - zeta*w)

In [162]:
def update(init_pred, init_prey1, init_prey12, alpha1, beta1, gamma1, 
           delta1, epsilon1, eta1, zeta1, step1, period1, pop_coef1, 
            shoot_deer, introduce_wolf):

    print(alpha1)
    # Initialize arrays for number of animals in populations
    pred = [init_pred]
    prey1 = [init_prey1]
    prey12 = [init_prey12]

    num_pred = init_pred
    num_prey1 = init_prey1
    num_prey12 = init_prey12
    
    period = period1
    pop_coeff = pop_coef1
#     day = 1
#     month = 30
#     year = 365

    # Update the number of animals time times and store them in arrays
    for n in range(time):
        dpred = pred_eq(gamma1, delta1, epsilon1, num_pred, num_prey1, num_prey12, step1)
        dprey1 = prey1_eq(alpha1, beta1, num_pred, num_prey1, step1)
        dprey12 = prey2_eq(eta1, zeta1, num_pred, num_prey12, step1)
        num_pred += dpred
        num_prey1 += dprey1
        num_prey12 += dprey12
        
        # Following a couple of real life events that can occur. To simulate an event
        # uncomment the event and run this cell again.
        
        # Regulation event 1. Every year prey regulation. 
        
        # Shoot percentage of the deer every period.
        if shoot_deer == "percentage" and (n % period == 0):
            num_prey1 *= pop_coeff
        
        # Shoot a set amount of the deer every period.
        elif shoot_deer == "static" and (n % period == 0) and (num_prey1 > pop_coeff):
            num_prey1 -= pop_coeff
        
        # Shoot a percentage of the growth every period.
        elif shoot_deer == "dynamic" and (n % period == 0):
            num_prey1 -= dprey1 * pop_coeff
        
        # gradual (monthly) introduction/increase of wolves.
        # Start with 1 wolf and comment 
        if introduce_wolf and (n % period == 0):
            num_pred += pop_coeff * 10
        
        pred.append(num_pred)
        prey1.append(num_prey1)
        prey12.append(num_prey12)

    # Plot the figure
    t = range(time+1)
    t = [step1*x for x in t]

    print(prey1[int(80*365)])
    
    plt.figure(figsize=(12,8))
    plt.plot(t, pred)
    plt.plot(t, prey1)
    plt.plot(t, prey12)
    plt.ylabel('Number of animals')
    plt.xlabel('Time (years)')
    plt.title('Wolf-Roe deer-Wild boar model', fontsize=20)
    plt.legend(['Wolf', 'Roe deer', 'Wild boar'], loc='upper left')
    plt.show()
    

ipred_slider = widgets.IntSlider(value=5, min=0, max=100, step=1)
iprey1_slider = widgets.IntSlider(value=45, min=0, max=5000, step=1)
iprey12_slider = widgets.IntSlider(value=20, min=0, max=300, step=1)
a1_slider = widgets.FloatSlider(value=alpha1, min=0, max=3, step=0.1)
b1_slider = widgets.FloatSlider(value=beta1, min=0, max=3, step=0.1)
g1_slider = widgets.FloatSlider(value=gamma1, min=0, max=3, step=0.1)
d1_slider = widgets.FloatSlider(value=delta1, min=0, max=3, step=0.1)
e1_slider = widgets.FloatSlider(value=epsilon1, min=0, max=3, step=0.1)
et1_slider = widgets.FloatSlider(value=eta1, min=0, max=3, step=0.1)
z1_slider = widgets.FloatSlider(value=zeta1, min=0, max=3, step=0.1)
s1_slider = widgets.FloatSlider(value=step1, min=0, max=0.005, step=0.0001)
period_slider = widgets.IntSlider(value=365, min=0, max=1095, step=1)
pop_coef_slider = widgets.FloatSlider(value=0.9, min=0, max=3, step=0.1)

interact_manual(update, init_pred=ipred_slider, init_prey1=iprey1_slider, init_prey12=iprey12_slider, 
                alpha1 = a1_slider, beta1 = b1_slider, gamma1 = g1_slider, delta1 = d1_slider, 
                epsilon1 = e1_slider, eta1 = et1_slider, zeta1 = z1_slider, step1=s1_slider, 
                period1 = period_slider, pop_coef1 = pop_coef_slider, 
                shoot_deer=["none", "percentage", "static", "dynamic"], introduce_wolf=False);

interactive(children=(IntSlider(value=5, description='init_pred'), IntSlider(value=45, description='init_prey1…

### Wild Boar and Wolf

In [3]:
# Wild boar and wolf
alpha2 = 6.22 # Birth rate prey 
beta2 = 12.4 / 182.75  # Death rate prey
gamma2 = 25/26 # Death rate predator
delta2 = 0.0356 # Grow rate predator

# Time * step = amount of years simulated
time2 = 1000000 
step2 = 0.0001

# Initial populations
init_pred2 = 1
init_prey2 = 500

In [4]:
def update(init_pred2, init_prey2, alpha2, beta2, gamma2, delta2, step2):
    pred2 = [init_pred2]
    prey2 = [init_prey2]

    num_pred2 = init_pred2
    num_prey2 = init_prey2

    for n in range(time):
        dpred2 = pred_eq(gamma2, delta2, num_pred2, num_prey2, step2)
        dprey2 = prey1_eq(alpha2, beta2, num_pred2, num_prey2, step2)
        num_pred2 += dpred2
        num_prey2 += dprey2
        pred2.append(num_pred2)
        prey2.append(num_prey2)

    t2 = range(time2+1)
    t2 = [step2*x for x in t2]

    plt.figure(figsize=(12,8))
    plt.plot(t2, pred2)
    plt.plot(t2, prey2)
    plt.ylabel('Number of animals')
    plt.xlabel('Time (years)')
    plt.title('Wolf-Wild boar model', fontsize=20)
    plt.legend(['Wolf', 'Wild boar'], loc='upper left')
    plt.show()

ipred2_slider = widgets.IntSlider(value=1, min=0, max=100, step=1)
iprey2_slider = widgets.IntSlider(value=500, min=0, max=300, step=1)
a2_slider = widgets.FloatSlider(value=6.22, min=0, max=1, step=0.01)
b2_slider = widgets.FloatSlider(value=12.4/182.75, min=0, max=1, step=0.01)
g2_slider = widgets.FloatSlider(value=25/26, min=0, max=1, step=0.01)
d2_slider = widgets.FloatSlider(value=0.0356, min=0, max=1, step=0.01)
s2_slider = widgets.FloatSlider(value=0.0001, min=0, max=10, step=0.001)

interact_manual(update, init_pred2=ipred2_slider, init_prey2=iprey2_slider, alpha2 = a2_slider,
                beta2 = b2_slider, gamma2 = g2_slider, delta2 = d2_slider, step2=s2_slider);

interactive(children=(IntSlider(value=1, description='init_pred2'), IntSlider(value=300, description='init_pre…

### Lotka-Volterra Vector Field (Stable Point Visualization) 

In [5]:
alpha3 = 9/11 # Birth rate prey 
beta3 = 48.6/365.25 # Death rate prey
gamma3 = 25/26 # Death rate predator
delta3 = 0.0356 # Grow rate predator

max_prey = 50 # Maximum begin population of prey
max_pred = 15 # Maximum begin population of predators

arrow_density = 2 # number of arrows per 1 unit

# Make a sample starting point
sample_pred = 7
sample_prey = 29
sample_length = 400
sample_step = 0.1
arrows = True
arrow_distance = 10

In [6]:
def update(alpha3, beta3, gamma3, delta3, max_prey, max_pred, arrow_density,
          sample_pred, sample_prey, sample_length, sample_step, arrows, arrow_distance):
    
    X, Y = np.meshgrid(np.arange(0, max_prey, 1/arrow_density), np.arange(0, max_pred, 1/arrow_density))

    stable_pred = alpha3 / beta3 # Stable initial population of predators
    stable_prey = gamma3 / delta3 # Stable initial population of prey
    print('The stable point (besides (0,0)) is at %.2f initial preys and %.2f initial predators.' % (stable_prey, stable_pred))

    # Check stability in formulas
    print('Predator difference (should be 0): %.2f' % pred_eq(gamma3, delta3, stable_pred, stable_prey, 1))
    print('Prey difference (should be 0): %.2f' % prey1_eq(alpha3, beta3, stable_pred, stable_prey, 1))

    U = np.zeros(X.shape)
    V = np.zeros(X.shape)

    for r in range(max_prey * arrow_density):
        for c in range(max_pred * arrow_density):
            U[c][r] = prey1_eq(alpha3, beta3, c/arrow_density, r/arrow_density, 1) 
            V[c][r] = pred_eq(gamma3, delta3, c/arrow_density, r/arrow_density, 1) 
            norm = np.sqrt((U[c][r])**2 + (V[c][r])**2)
            if norm != 0:
                U[c][r] = U[c][r] / norm
                V[c][r] = V[c][r] / norm

    s_prey = [sample_prey]
    s_pred = [sample_pred]
    for t in range(sample_length):
        cur_prey = s_prey[-1]
        cur_pred = s_pred[-1]
        s_prey.append(cur_prey + prey1_eq(alpha3, beta3, cur_pred, cur_prey, sample_step))
        s_pred.append(cur_pred + pred_eq(gamma3, delta3, cur_pred, cur_prey, sample_step))

    plt.figure(figsize=(15,14))
    plt.quiver(X, Y, U, V, scale=90, width=0.0015, color='b') # bigger scale is smaller arrows

    # Draw the sample line
    plt.plot(s_prey, s_pred, 'r-')

    # Draw starting point as a dot
    plt.plot(sample_prey, sample_pred, 'ro')

    # Draw arrows in line 
    if arrows:
        for i in range(9, sample_length, arrow_distance):
            if i+1 < sample_length:
                plt.arrow(s_prey[i], s_pred[i], s_prey[i+1]-s_prey[i], s_pred[i+1]-s_pred[i], color='r', head_width=0.2)

    plt.legend(["Example start point (%d prey, %d predators)" % (sample_prey, sample_pred)])
    plt.ylabel('Predator population')
    plt.xlabel('Prey population')
    plt.title('Vector Field of Lotka-Volterra equation', fontsize=20)
    plt.show()


a3_slider = widgets.FloatSlider(value=9/11, min=0, max=1, step=0.01)
b3_slider = widgets.FloatSlider(value=48.6/365.25, min=0, max=1, step=0.01)
g3_slider = widgets.FloatSlider(value=25/26, min=0, max=1, step=0.01)
d3_slider = widgets.FloatSlider(value=0.0356, min=0, max=1, step=0.01)
maxpred_slider = widgets.IntSlider(value=50, min=0, max=100, step=1)
maxprey_slider = widgets.IntSlider(value=15, min=0, max=300, step=1)

arrowdense_slider = widgets.IntSlider(value=2, min=0, max=10, step=1)

sampred_slider = widgets.IntSlider(value=7, min=0, max=100, step=1)
samprey_slider = widgets.IntSlider(value=29, min=0, max=100, step=1)
samplen_slider = widgets.IntSlider(value=400, min=0, max=1000, step=10)

sampstep_slider = widgets.FloatSlider(value=0.1, min=0, max=10, step=0.001)
arrowdist_slider = widgets.IntSlider(value=10, min=0, max=100, step=1)

interact_manual(update, alpha3=a3_slider, beta3=b3_slider, gamma3=g3_slider, delta3=d3_slider, 
                max_prey=maxprey_slider, max_pred=maxpred_slider, arrow_density=arrowdense_slider,
                sample_pred=sampred_slider, sample_prey=samprey_slider, sample_length=samplen_slider, 
                sample_step=sampstep_slider, arrows=True, arrow_distance=arrowdist_slider);

interactive(children=(FloatSlider(value=0.8181818181818182, description='alpha3', max=1.0, step=0.01), FloatSl…

In [7]:
r = np.array([9/11, 6.22, 25/26])
A = np.array([[1, 0, -48.6/365.25], [0, 1, ], []])